# CV Parser

## 1. Introducción

Este primer notebook tiene como objetivo acceder a un directorio para identificar aquellos archivos correspondientes a Curriculums, leerlos y guardar el texto en un formato semistructurado como JSON a través de un repositorio de MongoDB

## 2. Pasos

1. Obtención de la ruta de cada CV
2. Lectura PDFs - Itext
3. Identificación del lenguaje
4. Exportación a JSON - MongoDB

## 3. Código fuente

### 3.1 Obtención de la ruta de cada CV

El repositorio inicial se compone de tantas carpetas como usuarios con CV a analizar. Además, de cada usuario, se disponen de diversos ficheros, desde el CV, un extracto de notas hasta la fotocopia de algún documento personal.

In [1]:
import os
import glob

cvsDirectory = r"C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF"

Como existen diversos archivos e incluso diversos formatos de texto, se va a simplificar la solución buscando sólo aquellos documentos pdf que contengan una de las siguientes palabras clave: [cv, curriculum, vitae].

In [2]:
Keywords = ["cv","curriculum","vitae"]
PDFs = []
NonPDFs = 0

for folder in os.listdir(cvsDirectory):
    for f in glob.glob(cvsDirectory + "\\" + folder  + "\\*"):   
        if f[-4:].lower() == ".pdf":
            if any([keyword for keyword in Keywords if keyword in os.path.basename(f).lower()]):
                PDFs.append(cvsDirectory + "\\" + folder + "\\" + os.path.basename(f).lower())
        else:
            NonPDFs += 1

El número total de PDFs con los que se va a trabajar es: 

In [3]:
len(PDFs)

4613

### 3.2 Lectura PDFs - Itext

Una vez obtenido el path de cada CV, se va a proceder a su lectura utilizando la librería iText en su versión .Net (iTextSharp). Para poder invocar los métodos desarrollados en C#, se va a hacer uso de la librería CLR que permite la integración de librerías .NET en Python

In [4]:
import clr
clr.AddReference(r'C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\Itext\Itext-CVs.dll')

La librería está desarrollada bajo el namespace "wavespace", siendo la clase denominada "Pdf"

In [5]:
from wavespace import Pdf

Al igual que ser haría en .Net, se debe crear un objeto de la clase "Pdf" y, en este caso, configurar el encoding para poder leer los pdfs correctamente, utilizando la función "setEncoding"

In [6]:
pdfParser = Pdf()
pdfParser.setEncoding()

Una vez el objeto está cargado en memoria, se puede invocar su método principal, para la lectura del PDF. Para la lectura, cada PDF se va a guardar utilizando un array de Pandas

In [7]:
import pandas as pd
import time

start_time = time.time()
contents = []
types = []
files = []
pages = []

Por cada PDF:

- Se intenta abrir
- Se calcula el número de páginas
- Se lee cada página y se acumula el resultado en una variable de texto
  
Dos tipos de erroes son contemplados:

- El PDF no se puede abrir (está corrupto)
- Tras leer un PDF, no se obtiene nada. Para estos casos, vamos a suponer que es un PDF en formato imagen, ilegible por iText, y cuya lectura debe ser efectuada utilizando otra tecnología

In [8]:
for file in PDFs:
    
    if pdfParser.openPDF(file):
        
        content_ = ""
        pages_ = pdfParser.getNumberPages()+1
        for i in range(1,pages_):
            content_ += pdfParser.readPDFbyPage(i)
        
        pages.append(pages_)
        contents.append(content_)
        files.append(file)
        
        if content_ != "":
            types.append("Text")
        else:
            types.append("Image")
                
        pdfParser.closePDF()
    else:
        print("Error:" + str(file))    

CVs_Dataframe = pd.DataFrame()

CVs_Dataframe['CV'] = contents
CVs_Dataframe['File'] = files
CVs_Dataframe['Type'] = types
CVs_Dataframe['Pages'] = pages
    
print(time.time()-start_time)

Error:C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Carmen Diez Guijarro\carmen díez's cv.pdf
Error:C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Diana Morejón Hernandez\cv diana morejón hernández.pdf
Error:C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Diego Kreisler\cv - diego kreisler .pdf
Error:C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Diego Navas Arroyo\cv_diego_navas_actualizado.pdf
Error:C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Fernando Alvarez de Rivera\my cv7 (box).pdf
Error:C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Fernando Izquierdo\fernando izquierdo blanco cv inglés.pdf
Error:C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Fernando Izquierdo\fernando izquierdo cv español.pdf
Error:C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Gonzalo Puig\cv gonzalo puig.pdf
Error:C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Juan Salmador\cv+expediente.pdf
Error:C:\Users\fernando.c

In [9]:
TotalCVs = len(CVs_Dataframe)
TotalCVs_Text = len(CVs_Dataframe[CVs_Dataframe.Type == "Text"])
TotalCVs_Image = len(CVs_Dataframe[CVs_Dataframe.Type == "Image"])

In [10]:
print("Total PDFs - Text format: " + str(TotalCVs_Text) + " -> " + str(TotalCVs_Text/TotalCVs*100) + "%")
print("Total PDFs - Image format: " + str(TotalCVs_Image) + " -> " + str(TotalCVs_Image/TotalCVs*100) + "%")

Total PDFs - Text format: 4498 -> 97.78260869565217%
Total PDFs - Image format: 102 -> 2.2173913043478257%


### 3.3 Identificación del lenguaje

Como ya se predijo, los CVs pueden venir en distintos formatos e incluso en distintos idiomas. Es por ello, que se requiere de una clasificación inicial de dichos CVs en base al idioma, con el objetivo de descartar aquellos cuyo idioma no sea el castellano.

Para la detección del idoma se van a usar dos técnicas, una a través de NLTK y el conteo de stop words de diversos idiomas:

In [11]:
import nltk

def detect_language(text):
    
    languages_ratios = {}
    tokens = [word.lower() for word in nltk.word_tokenize(text)]
    
    for language in nltk.corpus.stopwords.fileids():
        stopwords_set = set(nltk.corpus.stopwords.words(language))
        words_set = set(tokens)
        common_elements = words_set.intersection(stopwords_set)

        languages_ratios[language] = len(common_elements)
    
    most_rated_language = max(languages_ratios, key=languages_ratios.get)
    return most_rated_language

Y una segunda utilizando la librería langdetect

In [12]:
from langdetect import detect

Cuando ambos algoritmos coinciden, vamos a considerar que el idioma ha sido correctamente detectado, de lo contrario, lo consideraremos indefinido

In [13]:
start_time = time.time()
languages = []

for index, row in CVs_Dataframe.iterrows():
    if isinstance(row['CV'], str):
        text = row['CV']
    else:
        text = row['CV'].decode("utf-8")

    language1 = detect_language(text)
    
    try:
        language2 = detect(text)
    except:
        language2 = ""

    if language1 == "spanish" and language2 == "es":
        languages.append("Spanish")
    elif language1 == "english" and language2 == "en":
        languages.append("English")
    else:
        languages.append("Undefined")           

CVs_Dataframe['Language'] = languages
print(time.time()-start_time)

389.433274269104


In [14]:
TotalCVs = len(CVs_Dataframe)
TotalCVs_Spanish = len(CVs_Dataframe[CVs_Dataframe.Language == "Spanish"])
TotalCVs_English = len(CVs_Dataframe[CVs_Dataframe.Language == "English"])
TotalCVs_Undefined = len(CVs_Dataframe[CVs_Dataframe.Language == "Undefined"])

In [15]:
print("Total PDFs - Spanish: " + str(TotalCVs_Spanish) + " -> " + str(TotalCVs_Spanish/TotalCVs*100) + "%")
print("Total PDFs - English: " + str(TotalCVs_English) + " -> " + str(TotalCVs_English/TotalCVs*100) + "%")
print("Total PDFs - Undefined: " + str(TotalCVs_Undefined) + " -> " + str(TotalCVs_Undefined/TotalCVs*100) + "%")

Total PDFs - Spanish: 3638 -> 79.08695652173913%
Total PDFs - English: 751 -> 16.326086956521742%
Total PDFs - Undefined: 211 -> 4.586956521739131%


### 3.4 Exportación a JSON - MongoDB

El último paso del notebook almacenará toda la información obtenida en un repositorio de MongoDB

In [16]:
from pymongo import MongoClient

In [17]:
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient["candidates"]
mycol = mydb["CV"]

In [18]:
mycol.insert_many(CVs_Dataframe.to_dict("records"))